In [3]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data

## 读取图片数据

In [5]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## weight、bias创建方法、正确率方法

In [6]:
# 
def weight_variable(shape):
    initail = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initail)

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = session.run(prediction, feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = session.run(accuracy, feed_dict={xs:v_xs, ys:v_ys, keep_prob:1})
    return result

> x 为输入的图片
> W 为weight
> sides第一位和最后一位是默认值，一定要为 1
> 中间两个1代表padding时在x方向运动一步，y方向运动一步，padding采用的方式是SAME

In [9]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [10]:
# pooling
# ksize第一位和第四位为是默认值 为 1
def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

## 定义变量和输入层数据


In [11]:
xs = tf.placeholder(tf.float32, [None, 784]) # 28*28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 转换成图片格式
# 1是channel的数量，因为我们输入的图片是黑白的，因此channel是1，例如如果是RGB图像，那么channel就是3
x_image = tf.reshape(xs, [-1, 28, 28, 1])

## 建立卷积层 
### conv1 layer 

In [12]:
# 
W_conv1 = weight_variable([5,5,1,32]) # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # outsize 28x28x32
h_pool1 = max_pool_2x2(h_conv1)                          # outsize 14x14x32

### conv2 layer

In [13]:
W_conv2 = weight_variable([5,5, 32, 64]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2) 

### func1 layer

In [14]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  #out size 7x7x64

### func2 layer

In [15]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)

## cross entropy 

In [16]:
# 损失熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),
                                             reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## session

In [17]:
session = tf.Session()
#初始化数据
init = tf.global_variables_initializer()

session.run(init)

## 训练

In [23]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    session.run(train_step, feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000],mnist.test.labels[:1000]))

0.106
0.73
0.842
0.877
0.892
0.912
0.918
0.918
0.933
0.932
0.945
0.951
0.951
0.952
0.953
0.96
0.962
0.965
0.965
0.964
0.966
0.968
0.962
0.965
0.968
0.972


KeyboardInterrupt: 